In [1]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd 
from path import Path

In [2]:
file_path = Path("../Resources/sanFranCrime.csv")
initial_df = pd.read_csv(file_path)
initial_df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


In [3]:
#create separate dataframe for location data
location_df = pd.DataFrame(data=initial_df, columns=["IncidntNum","Date","Address","X","Y","Location"])
location_df.head()

,IncidntNum,Date,Address,X,Y,Location
0,120058272,01/29/2016 12:00:00 AM,800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
1,120058272,01/29/2016 12:00:00 AM,800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
2,141059263,04/25/2016 12:00:00 AM,KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)"
3,160013662,01/05/2016 12:00:00 AM,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)"
4,160002740,01/01/2016 12:00:00 AM,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)"


In [4]:
#drop location data, date, dayofweek from initial_df
initial_df = initial_df.drop(columns=["Address","X","Y","Location","Date","DayOfWeek"])
initial_df.head()

,IncidntNum,Category,Descript,Time,PdDistrict,Resolution,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,11:00,SOUTHERN,"ARREST, BOOKED",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",11:00,SOUTHERN,"ARREST, BOOKED",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,14:59,BAYVIEW,"ARREST, BOOKED",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,23:50,TENDERLOIN,NONE,16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,00:30,MISSION,NONE,16000274071000


In [5]:
#remaining columns will be our features. Target column is resolution
initial_df.nunique()

IncidntNum    116699
Category          39
Descript         726
Time            1439
PdDistrict        10
Resolution        14
PdId          150500
dtype: int64

In [6]:
#get number of values for resolution column
initial_df["Resolution"].value_counts()

NONE                                      107780
ARREST, BOOKED                             39416
UNFOUNDED                                   1608
JUVENILE BOOKED                             1056
EXCEPTIONAL CLEARANCE                        371
ARREST, CITED                                144
CLEARED-CONTACT JUVENILE FOR MORE INFO        58
NOT PROSECUTED                                22
LOCATED                                       20
PSYCHOPATHIC CASE                             17
JUVENILE CITED                                 3
COMPLAINANT REFUSES TO PROSECUTE               2
JUVENILE DIVERTED                              2
PROSECUTED BY OUTSIDE AGENCY                   1
Name: Resolution, dtype: int64

In [7]:
#bucket everything besides NONE and arrest,booked for now to create binary classification

#bucket resolutions
resolution_counts = initial_df["Resolution"].value_counts()

replace_resolution = list(resolution_counts[resolution_counts<1700].index)

for resolution in replace_resolution:
    initial_df["Resolution"] = initial_df["Resolution"].replace(resolution, "other")
    
initial_df["Resolution"].value_counts()

NONE              107780
ARREST, BOOKED     39416
other               3304
Name: Resolution, dtype: int64